In [ ]:
#| default_exp methods

In [ ]:
#| hide
#| export
from nbdev.showdoc import *
import fastcore.test
from gh_pages_example.utils import *
from gh_pages_example.types import *
import typing

import numpy as np
import nptyping

In [ ]:
np.set_printoptions(suppress = True)

# Methods in Evolutionary Game Theory

> A set of methods for solving Evolutionary Games (see Nowak 2006 and the references section)

## Evolutionary Dynamics in Finite Populations

We examine a finite population of players using different strategies who engage in social learning.

In the limit of small mutations, most of the time everyone plays the same strategy. States in which everyone plays the same strategy are known as **monomorphic states**. Occassionally, mutant strategies can fixate in the population, resulting in everyone adopting the same new strategy. We can use Markov Chains to analyse the relative frequencies with which each strategy is played by the population.

The steps for computing the ergodic (i.e. long-run, stationary) strategy distribution is as follows:

1. Build a transition matrix between all monomorphic states
2. Find the ergodic distribution for the markov chain defined using this transition matrix

### Fermi social learning

> A Fermi social learning rule means that individuals make pairwise comparisons between their own strategy and and another strategy in the population that they may choose to copy.

#### Derivation

Each period of the evolutionary game involves individuals being randomly selected to play against one another individual.

Letting $Z$ denote the size of the population, and $π$ denote the game's payoff matrix, we can compute the fitness of a strategy, $B$ for example, when $k$ individuals are of type $B$ as follows:

\begin{equation}
ΠB_k = πBA \frac{k-1}{Z - 1} + πBB \frac{Z-k}{Z- 1}
\end{equation}

where $πBA$ and $πBB$ are the payoffs for playing $B$ against type $A$ or $B$ respectively.

The **Fermi social learning rule** adopts strategy $B$ selected from the population over their current strategy $A$ with probability given by:

\begin{equation}
Pr(adopt \, B | k) = \frac{1}{(1 + \exp^{-\beta (ΠB_k - ΠA_k)})}
\end{equation}

where $ΠB_k - ΠA_k$ is the relative fitness of strategy $B$ over $A$ in a population with $k$ individuals of type $B$, the rest of type $A$. Notice how the larger the relative fitness, the closer the denominator, and therefore the probability, is to $1$.

Using the Fermi social learning rule above, we can write the probability of increasing the number of type $B$ individuals as

\begin{equation}
T^+_B(k) = \frac{Z-k}{Z} \frac{k}{Z} Pr(adopt \, B | k) 
\end{equation}
Z
as an individual of type $A$ needs to randomly be chosen to compare their strategy against someone of type $B$.

and the probability of decreasing the number of type $B$ individuals as

\begin{equation}
T^-_B(k) = \frac{k}{Z} \frac{Z-k}{Z} Pr(adopt \, A | k) 
\end{equation}

as an individual of type $B$ needs to randomly be chosen to compare their strategy against someone of type $A$.

We will often employ their ratio, which is: 

\begin{equation}
\frac{T^-_B(k)}{T^+_B(k)} = \frac{Pr(adopt \, A | k) }{Pr(adopt \, B | k)} = \frac{1 + \exp^{-\beta (ΠB_k - ΠA_k)}}{1 + \exp^{-\beta (ΠA_k - ΠB_k)}}
\end{equation}

Notice that $\frac{1 + \exp^x}{1 + \exp^{-x}} = \exp^{x}$

So, this ratio simplifies to $\frac{T^-_B(k)}{T^+_B(k)} =  \exp^{-\beta (ΠB_k - ΠA_k)}$


#### Definition

In [ ]:
#| export

def fermi_learning(fitnessA:nptyping.NDArray, # fitness of strategy A
                   fitnessB:nptyping.NDArray, # fitness of strategy B
                   β:nptyping.NDArray, # learning rate
                  ) -> nptyping.NDArray:
    """Compute the likelihood that a player with strategy A adopts strategy B using the fermi function."""
    return (1 + np.exp(-β*(fitnessB - fitnessA)))**(-1)

#### Examples and Tests

When each strategy has the same fitness, then the likelihood that a player adopts strategy $B$ is 50%, no matter the value of $\beta$.

In [ ]:
x = fermi_learning(np.array([5]),
                   np.array([5]),
                   np.array([1]),)
nptyping.assert_isinstance(x, nptyping.NDArray[nptyping.Shape["1"], typing.Any])
fastcore.test.test_eq(x, 0.5)

### Fixation rate

> The fixation rate for type B in a population of type A, $\rho$, is defined as the probability that the appearance of a mutant of type B leads to the entire population adopting type B instead of A, i.e. what is the likelihood that a mutant of type B invades population A.

#### Derivation


A derivation of the fixation rate defined below can be found in Nowak 2006 (reproduced below).

> Consider a one-dimensional stochastic process on a discrete state space, $ i \in \{0, 1, \cdots, N\}$ that represents the number of individuals in a population of $N$ individuals who are of type $B$, the rest are type $A$.
>
> In each stochastic event, the number of individuals of type $B$ can at most increase or decrease by 1.
>
> For a given number of individuals, $i$, let $a_i$, $b_i$, and $1 - a_i - b_i$ represent the chance of an increase, decrease, or no change in $i$.
> 
> This stochastic process follows the transition matrix ,$P$ (*not to be confused with the transition matrices we discuss elsewhere!*)
>
>
> \begin{equation}
P \, = \, \begin{pmatrix}
1 & 0 & 0 & \cdots & 0 & 0 & 0\\
b_1 & (1 - a_1 - b_1) & a_1 & \cdots & 0 & 0 & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & b_{n-1} & (1 - a_{n-1} - b_{n-1}) & a_{n-1}\\
0 & 0 & 0 & \cdots & 0 & 0 & 1\\
\end{pmatrix}
\end{equation}
>
> Denote by $x_i$ the probability of reaching state $N$ when starting from $i$.
>
> From transition matrix $P$ above, we can see that $x_i$ must satisfy:
>
> $x_0 = 0$
>
> $x_i = b_i x_{i-1} + (1 - a_i - b_i) x_i + a_i x_{i+1}$
>
> $x_N = 1$
>
> The fixation rate for a mutant B in a population of type A is clearly $x_1$
>
> We can solve for $x_i$ by rewriting the above as $b_i x_i - b_i  x_{i-1} = a_i x_{i+1} - a_i x_i$.
> 
> We can denote $y_i = x_i - x_{i-1}$ to simplify the above to $y_{i+1} = \frac{b_i}{a_i} y_i$
>
> Notice that $\sum_{i=1}^N{y_i} = x_N - x_0 = 1$ and that $y_1 = x_1$
>
> We can use the above to write
\begin{equation}
x_1 + {\sum_{i=2}^N{y_i}} = x_1 (1 + {\sum_{i=1}^{N-1}{\prod_{j=1}^{i} \frac{b_j}{a_j}}}) = 1
\end{equation}
>
> And so
\begin{equation}
x_1 = \frac{1}{(1 + \sum_{i=1}^{N-1}{\prod_{j=1}^{i} \frac{b_j}{a_j}})}
\end{equation}
>
> Note that $x_1$ is the fixation rate for a mutant $B$ in a population of type $A$, often denoted as $\rho$.
>
> *Also note that $x_{N-1}$ is the fixation rate for a mutant $A$ in a population of type $B$. We could find expressions for all $x_i$ if we note that $x_i = x_1 (1 + \sum_{j=1}^{i-1}{\prod_{k=1}^{j} \frac{b_k}{a_k}})$ (see Nowak 2006 for further details).*

#### Definition

In [ ]:
#| export

T_type = list[nptyping.NDArray[nptyping.Shape["N_models"], typing.Any]]

def fixation_rate(Tplus: T_type, # A list of NDarrays, one array (of size n_models) for each possible number of mutants in the population; the probability of gaining one mutant
                  Tneg: T_type, # A list of NDarrays, one array (of size n_models) for each possible number of mutants in the population; the probability of losing one mutant
                 ) -> nptyping.NDArray[nptyping.Shape["N_models"], typing.Any]: # Fixation rates for the given strategy in each model
    """Calculate the likelihood that a mutant invades the population."""
    Z = len(Tplus) + 1
    ρ = (np.sum([np.prod([Tneg[j-1]/Tplus[j-1]
                         for j in range(1,i+1)],
                        axis=0,
                        keepdims=False)
                 for i in range(1,Z)],
                axis=0,
                keepdims=False)
        + 1)**-1
    return ρ

In [ ]:
show_doc(fixation_rate)

---

[source](https://github.com/PaoloBova/gh-pages-example/blob/main/gh_pages_example/methods.py#L28){target="_blank" style="float:right; font-size:smaller"}

### fixation_rate

>      fixation_rate (Tplus:list[nptyping.base_meta_classes.NDArray],
>                     Tneg:list[nptyping.base_meta_classes.NDArray])

Calculate the likelihood that a mutant invades the population.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| Tplus | list | A list of NDarrays, one array (of size n_models) for each possible number of mutants in the population; the probability of gaining one mutant |
| Tneg | list | A list of NDarrays, one array (of size n_models) for each possible number of mutants in the population; the probability of losing one mutant |
| **Returns** | **NDArray** | **Fixation rates for the given strategy in each model** |

#### Examples and Tests

When the chance of gaining a mutant always equals the chance of losing a mutant, then the fixation rate will be $\frac{1}{Z}$

Note that because we have to sample the population for a mutant and the player of the type being invaded, the chance of gaining or losing a mutant can be no greater than $\frac{k}{Z} \frac{Z-k}{Z}$

In [ ]:
Z = 2 # With Z=2, we only need to evaluate Tplus and Tneg for when k=1
Tplus_example = [np.array([1/8])]
Tneg_example =  [np.array([1/8])]

In [ ]:
#|  hide
# validate test inputs
assert len(Tplus_example) == len(Tneg_example)
for tplus, tneg in zip(Tplus_example, Tneg_example):
    assert tplus.shape == tneg.shape

In [ ]:
fixation_rate_result = fixation_rate(Tplus_example, Tneg_example)

In [ ]:
#| hide
nptyping.assert_isinstance(fixation_rate_result,
                           nptyping.NDArray[nptyping.Shape["1"], typing.Any])

True

In [ ]:
fastcore.test.test_eq(fixation_rate_result, np.array([0.5]))

When the chance of gaining a mutant is half the chance of losing a mutant, then the fixation rate will be

\begin{equation}
\rho = \frac{1}{(1 + \sum_{j=1}^{Z-1}{2^j})}
\end{equation}

When $Z=2$, we have $\rho = \frac{1}{3}$

In [ ]:
Z = 2 # With Z=2, we only need to evaluate Tplus and Tneg for when k=1
Tplus_example = [np.array([0.1])]
Tneg_example =  [np.array([0.2])]

In [ ]:
#|  hide
# validate test inputs
assert len(Tplus_example) == len(Tneg_example)
for i, tplus in enumerate(Tplus_example):
    assert tplus.shape == Tneg_example[i].shape

In [ ]:
fixation_rate_result = fixation_rate(Tplus_example, Tneg_example)

In [ ]:
fastcore.test.test_eq(fixation_rate_result, np.array([1/3]))

In [ ]:
#| hide
nptyping.assert_isinstance(fixation_rate_result,
                           nptyping.NDArray[nptyping.Shape["1"], typing.Any])

True

We could instead consider an example where we have a mutant Defector (D) who appears in a population of Cooperators (C) playing a standard Prisoner's Dilemma.

We will consider an example of such a scenario where chance of gaining/losing a D player be given by $\frac{1}{1 + e^{\pm \beta \frac{Z+1}{Z-1}}}$.

The fixation rate will be given by the following expression:

\begin{equation}
\rho = \frac{1}{1 + \sum_{j=1}^{Z-1}{(\frac{1 + e^{- \beta \frac{Z+1}{Z-1}}}{1 + e^{\beta \frac{Z+1}{Z-1}}})^j}}
\end{equation}

For this example, we will let $\beta=1$ and $Z=10$, so $\beta \frac{Z+1}{Z-1} = \frac{11}{9}$.

In [ ]:
β = 1
Z = 10
ρ_CD = 1 / (1 + sum((1 + np.exp(- β * (Z + 1) / (Z-1)))**j 
                    / (1 + np.exp(β * (Z + 1) / (Z-1)))**j
                    for j in range(1, Z)))
Tplus_example = [np.array([1 / (1 + np.exp(- β * (Z + 1) / (Z-1)))])
                 for _ in range(Z-1)]
Tneg_example =  [np.array([1 / (1 + np.exp(β * (Z + 1) / (Z-1)))])
                 for _ in range(Z-1)]

In [ ]:
#|  hide
# validate test inputs
assert len(Tplus_example) == len(Tneg_example)
for i, tplus in enumerate(Tplus_example):
    assert tplus.shape == Tneg_example[i].shape

In [ ]:
#| hide
nptyping.assert_isinstance(fixation_rate(Tplus_example, Tneg_example),
                           nptyping.NDArray[nptyping.Shape["1"], typing.Any])

True

In [ ]:
fastcore.test.is_close(fixation_rate(Tplus_example, Tneg_example), ρ_CD)

True

Finally, it is useful to know how the fixation rate behaves when any elements of Tplus are zero (as the fixation rate divides by those elements). Even though the Fermi learning rule we use theoretically gives a number between 0 and 1 exclusive, in practise the number may underflow to a 0 if low enough. This will cause unexpected behaviour if we allow it in our alogorithm for computing the transition matrix.

We can avoid this issue by using a slightly altered method for calculating the fixation rate, taking advantage of our choice to use the `fermi_learning` rule.

In the above fixation rate calculations we used the `fermi_learning` function to calculate the probability of a player with strategy $D$ adopting strategy $C$ (and likewise for the probability of a player with $C$ adopting $D$). Their ratio takes the form, $\frac{1 + e^x}{1 + e^{-x}}$. It is not too hard to verify that $\frac{1 + e^x}{1 + e^{-x}} = e^x$.

Moreover, we can avoid taking the product of the ratios at all, since the product of exponentials (with the same base) is just the exponential of the sum of their exponents.

By using the above substitution and algebraic manipulation, we can substantially mitigate the numerical stability issues. For this reason, we will not use `fermi_learning` nor `fixation_rate` in our algorithm at all (although in most cases we would expect these methods to yield the same answers). Instead, we will use `fixation_rate_stable`.

In [ ]:
#| export
def fixation_rate_stable(ΠA:list, # Average payoffs for the strategy A they consider adopting for each number of mutants following A
                         ΠB:list, # Average payoffs for the strategy B that the player currently follows for each number of mutants following A
                         β:Array1D, # learning rate 
                        ):
    """Calculate the likelihood that a mutant B invades population A
    using a numerically stable method."""
    fastcore.test.test_eq(len(ΠA), len(ΠB))
    Z = len(ΠA) + 1
    ρ = (np.sum([np.exp(np.clip(np.sum([-β*(ΠB[j-1] - ΠA[j-1])
                                        for j in range(1,i+1)],
                                       axis=0,
                                       keepdims=False),
                                -500,
                                500)) # avoid underflow/overflow warnings
                 for i in range(1,Z)],
                axis=0,
                keepdims=False)
        + 1)**-1
    return ρ

We can see in the examples which follow that both methods usually give the same answers.

To match an earlier example where `Tplus` and `Tneg` were both equal to $\frac{1}{8}$ (as $Z=2$ we only need to consider one value for each when $k=1$), we let $\beta=1$ and recall that $T^+_B(k) = \frac{Z-k}{Z} \frac{k}{Z} Pr(adopt \, B | k) = \frac{Z-k}{Z} \frac{k}{Z} \frac{1}{1 + \exp^{-\beta (ΠB(k) - ΠA(k))}} $

We can then say that $ΠA - ΠB = \log{(\frac{1}{\frac{4}{8}} - 1)} = \log{\frac{4}{4}} = \log{4} - \log{4}$

Notice that to achieve netural drift, the payoffs have to be equal.

In [ ]:
Z = 2
β = 1
ΠA = [np.array([np.log(4)])]
ΠB = [np.array([np.log(4)])]
result = fixation_rate_stable(ΠA, ΠB, β)
fastcore.test.test_close(result, 0.5)

We can also consider an example from a payoff matrix I've run into in practise.

In [ ]:
payoffs = np.array([[51, 0.6, 51],
                    [114.3, 57.75, 39.38],
                    [51, 0.99798, 51]])

We are interested in the fixation rate of a mutant B in a population of A

Strategy A is the strategy represented by row 3

Strategy B is the strategy represented by row 2


In [ ]:
Z = 100
β = 1

We need only the average payoffs for the stable calculation.

In [ ]:
ΠA = [k/(Z-1) * payoffs[2,1] + (Z-k-1)/(Z-1) * payoffs[2,2]
      for k in range(1, Z)]
ΠB = [(k-1)/(Z-1) * payoffs[1,1] + (Z-k)/(Z-1) * payoffs[1,2]
      for k in range(1, Z)]

result_stable = fixation_rate_stable(ΠA, ΠB, β)

We also need the adoption rates for the unstable calculation

In [ ]:
Tneg = [fermi_learning(ΠB[k-1], ΠA[k-1], β)
                     for k in range(1, Z)]
Tplus = [fermi_learning(ΠA[k-1], ΠB[k-1], β)
         for k in range(1, Z)]

# Naiive and unstable calculation
result_unstable = fixation_rate(Tplus, Tneg)

In [ ]:
fastcore.test.test_close(result_stable, 0)
fastcore.test.test_close(result_unstable, 0)

### Build transition matrix

Recall that step 1 of finding the solution to the Evolutionary Game dynamics is to build a transition matrix between all monomorphic states. 

The transition matrix captures the probability that if the population of the Evolutionary Game transitions to another state. We read an entry of the transition matrix as saying the probability of transitioning from the row state to column state.

In [ ]:
#| export
class ModelTypeEGT():
    """This is the schema for an Evolutionary Game Theory model.
    
    Note: This schema is not enforced and is here purely for documentation
    purposes."""
    def __init__(self, 
                 Z: int, # the size of the population
                 strategy_set: list[str], # the set of strategies in the model
                 β: Array1D, # the learning rate
                 payoffs: Array3D, # the payoffs of the game
                 transition_matrix: Array3D=None, # the model's transition matrix
                 ergodic: Array2D=None, # ergodic distribution of the model's markov chain
                ):
        pass

In [ ]:
show_doc(ModelTypeEGT)

---

[source](https://github.com/PaoloBova/gh-pages-example/blob/main/gh_pages_example/methods.py#L65){target="_blank" style="float:right; font-size:smaller"}

### ModelTypeEGT

>      ModelTypeEGT (Z:int, strategy_set:list[str],
>                    β:gh_pages_example.types.Array1D,
>                    payoffs:gh_pages_example.types.Array3D,
>                    transition_matrix:gh_pages_example.types.Array3D=None,
>                    ergodic:gh_pages_example.types.Array2D=None)

This is the schema for an Evolutionary Game Theory model.

Note: This schema is not enforced and is here purely for documentation
purposes.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| Z | int |  | the size of the population |
| strategy_set | list |  | the set of strategies in the model |
| β | Array1D |  | the learning rate |
| payoffs | Array3D |  | the payoffs of the game |
| transition_matrix | Array3D | None | the model's transition matrix |
| ergodic | Array2D | None | ergodic distribution of the model's markov chain |

In [ ]:
#| export
def build_transition_matrix(models:dict # A dictionary that contains the parameters in `ModelTypeEGT`
                           ):
    """Build a transition matrix between all monomorphic states
    using the fermi social learning rule for each model.    
    """
    
    Z, S, β = [models[k] for k in ['Z','strategy_set', 'β']]
    π = models['payoffs']
    n_models = π.shape[0]
    M = np.zeros(( n_models, len(S), len(S)))
    for row_ind, s in enumerate(S):
        for col_ind, sₒ in enumerate(S):
            if row_ind == col_ind:
                M[:, row_ind, row_ind] += 1
                # We calibrate these entries later so rows add up to 1
                continue
            πAA = π[:, row_ind, row_ind]
            πAB = π[:, row_ind, col_ind]
            πBA = π[:, col_ind, row_ind]
            πBB = π[:, col_ind, col_ind]
            ΠA = [πAA*(Z-k-1)/(Z-1) + πAB*k/(Z-1)
                  for k in range(1, Z)]
            ΠB = [πBA*(Z-k)/(Z-1)  + πBB*(k-1)/(Z-1)
                  for k in range(1, Z)]
            # We use a numerically stable method to find the fixation rate, ρ.
            # ρ is the probability that mutant B successfully invades A
            ρ = fixation_rate_stable(ΠA, ΠB, β)
            # A less stable but still valid method is as follows:
            # Tneg = [fermi_learning(ΠB[k-1], ΠA[k-1], β)
            #         for k in range(1, Z)]
            # Tplus = [fermi_learning(ΠA[k-1], ΠB[k-1], β)
            #          for k in range(1, Z)]
            # ρ = fixation_rate(Tplus, Tneg)
            M[:, row_ind, col_ind] = ρ / max(1, len(S)-1)
            M[:, row_ind, row_ind] -= ρ / max(1, len(S)-1)
    return {**models, "transition_matrix": M}

#### Examples and Tests

Consider the following two examples.

**Example 1**

Let all payoffs be equal in the game's payoff matrix. All expected payoffs will be equal too.

So, Fermi learning will say that each individual has a 50% chance of adopting the behaviour of the one they observe.

We therefore have an equal chance during each epoch of gaining or losing an individual of the given type, in this example we denote the type as $s \in \{A, B\}$, although this probability depends on population size $Z$ and the current number of individuals of that type, $k$, $T^+_s(k) = T^-_s(k) = \frac{Z-k}{Z} \frac{k}{Z} \frac{1}{2}$.

Recall that we calculate the fixation rate, $\rho$ as follows:
\begin{equation}
\rho = \frac{1}{1 + \sum_{j=1}^{N-1}{\prod_{k=1}^{j} \frac{b_k}{a_k}}}
\end{equation}
where $N=Z$, $b_k = T^-_s(k)$ and $a_k = T^+_s(k)$

In this example, for each strategy $s$, $T^-_s(k) = T^+_s(k), \, \forall k$, so $\rho = \frac{1}{Z}$.

We only have $2$ strategies, and $Z=10$, so the final transition matrix will look like

\begin{equation}
M \, = \, \begin{pmatrix}
1 - \frac{\rho}{2 - 1} & \frac{\rho}{2 - 1} &\\
\frac{\rho}{2 - 1} & 1 - \frac{\rho}{2 - 1}\\
\end{pmatrix}
= \begin{pmatrix}
0.9 & 0.1 &\\
0.1 & 0.9\\
\end{pmatrix}
\end{equation}

Note that the above example describes neutral drift, the idea that even if there is no advantage to be gained from any particular strategy, social learning can still result in the spread of that behaviour. Neutral drift also occurs if we set the Fermi learning rate $\beta = 0$, no matter what payoff matrix describes the game.

In [ ]:
payoffs = np.array([[[2, 2],
                     [2, 2]]
                   ])
Z = 10
β = 1
models = {"payoffs": payoffs,
          "Z": Z,
          "β": β,
          "strategy_set": ["A", "B"],
         }
result = build_transition_matrix(models)

In [ ]:
fastcore.test.test_close(result['transition_matrix'],
                         np.array([[0.9, 0.1],
                                   [0.1, 0.9]]))

**Example 2**

Let the payoff matrix be akin to a Prisoner's Dilemma with two strategies, $C$ or $D$ (Cooperate or Defect respectively):

\begin{pmatrix}
2 & 0\\
3 & 1\\
\end{pmatrix}

Again, for this simple example, the relative average success of strategy $C$ is independent of the number of $C$ players, $k$. This is rarely the case in practise but permits a more legible example.

$C$'s relative success over $D$ will be $\frac{2 (k-1)}{Z-1} - \frac{3 k + (Z - k - 1)}{Z-1} = - \frac{Z + 1}{Z-1}$.

Fermi learning means the probability of a $D$ player adopting what they see $C$ do is:

\begin{equation}
\frac{1}{1 + e^{- \beta (\Pi_C(k) - \Pi_D(k))}} = \frac{1}{1 + e^{\beta \frac{Z + 1}{Z-1}}}
\end{equation}

The fixation rate for mutant $C$ in a population of $D$ players, $\rho_{DC}$, can be computed as

\begin{equation}
\rho_{DC} = \frac{1}{1 + \sum_{j=1}^{Z-1}{(\frac{1 + e^{\beta \frac{Z + 1}{Z-1}}}{1 + e^{-\beta \frac{Z + 1}{Z-1}}})^j}}
\end{equation}

Similarly, the fixation rate for mutant $D$ in a population of $C$ players, $\rho_{CD}$, can be computed as 

\begin{equation}
\rho_{CD} = \frac{1}{1 + \sum_{j=1}^{Z-1}{(\frac{1 + e^{-\beta \frac{Z + 1}{Z-1}}}{1 + e^{\beta \frac{Z + 1}{Z-1}}})^j}}
\end{equation}

For $Z=10$ and $\beta = 1$, the above yields the following transition matrix,

\begin{equation}
M \, = \, \begin{pmatrix}
1 - \frac{\rho_{CD}}{2 - 1} & \frac{\rho_{CD}}{2 - 1} &\\
\frac{\rho_{DC}}{2 - 1} & 1 - \frac{\rho_{DC}}{2 - 1}\\
\end{pmatrix}
\approx \begin{pmatrix}
0.295 & 0.705 &\\
0.000 & 1.000\\
\end{pmatrix}
\end{equation}



Note how in the above fixation rate calculations how we used the `fermi_learning` function to calculate the probability of a player with strategy $D$ adopting strategy $C$ (and likewise for the probability of a player with $C$ adopting $D$). This function has special properties which aid us in calculating the fixation rate.

Notice how the ratio of the two adoption rates takes the form, $\frac{1 + e^x}{1 + e^{-x}}$. It is not too hard to verify that $\frac{1 + e^x}{1 + e^{-x}} = e^x$.

We utilities this property to considerably improve the numerical stability of our algorithm for building a transition matrix. For this reason, we do not use `fermi_learning` in our algorithm at all.

We can similarly note that $\frac{1}{1 + e^{-x}} = 1 - \frac{1}{1 + e^{x}}$, i.e. the two adoption rates are complementary probabilities.


In [ ]:
payoffs = np.array([[[2, 0],
                     [3, 1]],
                   ])
Z = 10
β = 1
models = {"payoffs": payoffs,
          "Z": Z,
          "β": β,
          "strategy_set": ["C", "D"],
         }
result = build_transition_matrix(models)

In [ ]:
ρ_CD = 1 / (1 + sum((1 + np.exp(- β * (Z + 1) / (Z-1)))**j 
                    / (1 + np.exp(β * (Z + 1) / (Z-1)))**j
                    for j in range(1, Z)))
ρ_DC = 1 / (1 + sum((1 + np.exp(β * (Z + 1) / (Z-1)))**j
                    / (1 + np.exp(- β * (Z + 1) / (Z-1)))**j 
                    for j in range(1, Z)))

In [ ]:
ρ_CD_alt = 1 / (1 + sum(np.exp(- j * β * (Z + 1) / (Z-1))
                        for j in range(1, Z)))
ρ_DC_alt = 1 / (1 + sum(np.exp(j * β * (Z + 1) / (Z-1))
                        for j in range(1, Z)))

In [ ]:
fastcore.test.test_close(ρ_CD, ρ_CD_alt)
fastcore.test.test_close(ρ_DC, ρ_DC_alt)

In [ ]:
fastcore.test.test_close(result['transition_matrix'],
                         np.array([[1- ρ_CD, ρ_CD],
                                   [ρ_DC, 1 - ρ_DC]]))

#### Example 3

Here is an additional example for the 3 by 3 matrix we discussed when testing other functions.

This time, we make sure we get the correct probabilities for each transition.

In [ ]:
payoffs = np.array([[[51, 0.6, 51],
                     [114.3, 57.75, 39.38],
                     [51, 0.99798, 51]],
                   ])

In [ ]:
expected = np.array([[[0.495, 0.5, 0.005],
                     [0, 1, 0],
                     [0.005, 0, 0.995]],
                   ])

In [ ]:
Z = 100
β = 1
models = {"payoffs": payoffs,
          "Z": Z,
          "β": β,
          "strategy_set": ["AS", "AU", "PS"],
         }
result = build_transition_matrix(models)
fastcore.test.test_close(result['transition_matrix'], expected)

In [ ]:
show_doc(build_transition_matrix)

---

[source](https://github.com/PaoloBova/gh-pages-example/blob/main/gh_pages_example/methods.py#L81){target="_blank" style="float:right; font-size:smaller"}

### build_transition_matrix

>      build_transition_matrix (models:dict)

Build a transition matrix between all monomorphic states
using the fermi social learning rule for each model.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| models | dict | A dictionary that contains the parameters in `ModelTypeEGT` |

### Find ergodic strategy distribution

Step 2 is to find the ergodic distribution for the Evolutionary Game using the transition matrix we constructed in step 1.

Let $M$ denote the transition matrix, and $\omega_t$ be the column vector describing the proportions with which each strategy is played in the population.

We can describe the evolution of this system with $\omega_{t+1} = M^T \omega_t$, i.e. the proportion of players that use a given strategy in the next round will be equal to the sum of the proportions of players for each strategy who adopted that strategy in the current round. Equivalently, we can also consider $\omega_t$ as describing the probabilities that the system at time t is in each of the monomorphic states.

As each of the monomporphic states described in the transition matrix is reachable from any other with some probability and since the transition probabilities only depend on the current state, what we have is a markov chain which is irreducible.

The ergodicity theorem guarantees that such irreducible and aperiodic markov chains have an ergodic distribution that the system converges to, no matter where it starts. An ergodic distribution (also called a stationary distribution),  $\omega^*$ satisfies  $\omega^* = M^T \omega^*$ [[1]](https://gregorygundersen.com/blog/2019/10/28/ergodic-markov-chains/) [[2]](http://www.stat.columbia.edu/~liam/teaching/neurostat-spr11/papers/mcmc/Ergodicity_Theorem.pdf) [[3]](https://textbooks.math.gatech.edu/ila/1553/stochastic-matrices.html).

Our ergodic distribution, $\omega^*$, is therefore defined as the normalised right-hand eigenvector with eigenvalue 1 of the transposed transition matrix, $M^T$ (or equivalently, if we defined $\omega$ as a row vector instead, $\omega^*$ would be the left-hand eigenvector with eigenvalue 1 of transition matrix, $M$; numerical computing packages usually return the right-hand eigenvectors more directly, which is why I used the other formalism).

We use standard linear algebra methods from the [numpy](https://numpy.org/) package to find this eigenvector. These numerical methods will usually not return an eigenvector which is normalised to sum to 1, so we must normalise the eigenvector we are given. See their documentation to learn more about these numerical methods.

In [ ]:
#| export
def find_ergodic_distribution(models:dict # A dictionary that contains the parameters in `ModelTypeEGT`
                             ):
    """Find the ergodic distribution of a markov chain with the
    given transition matrix."""
    M = models["transition_matrix"]
    # find unit eigenvector of markov chain
    Λ,V = np.linalg.eig(M.transpose(0,2,1))
    V = np.real_if_close(V)
    x = np.isclose(Λ, 1)
    # if multiple unit eigenvalues then choose the first
    y = np.zeros_like(x, dtype=bool)
    idx = np.arange(len(x)), x.argmax(axis=1)
    y[idx] = x[idx]
    ergodic = np.array(V.transpose(0,2,1)[y], dtype=float)
    # ensure ergodic frequencies are positive and sum to 1
    ergodic = np.abs(ergodic) / np.sum(np.abs(ergodic), axis=1)[:, None]
    return {**models, 'ergodic':ergodic}

#### Examples and Tests

Let our transition matrix, $M$ be

\begin{equation}
M = \begin{pmatrix}
\frac{3}{4} & \frac{1}{4} \\
\frac{1}{4} & \frac{3}{4} \\
\end{pmatrix}
\end{equation}

Note that $M^T$ is a stochastic matrix because each column of the transposed matrix would sum to $1$ (in general the rows of the transposed matrix are unlikely to sum to 1, but choosing an example like the above makes it easy to compute the eigenvectors).

It's not too hard to verify that the characteristic polynomial of $M^T$ can be factored into $(\lambda - 1)(\lambda - \frac{1}{2})$, so we have two eigenvalues, $1$ and $\frac{1}{2}$.

It's not too hard to verify that column vector $[1, 1]$ is the eigenvector of $M^T$ with eigenvalue $1$
.

Now that we know the weights placed on each strategy, we can compute the strategy distribution by normalising our eigenvector.

The ergodic distribution i $\omega^* = [\frac{1}{2}, \frac{1}{2}]$.

In [ ]:
M = np.array([[[3/4, 1/4],
               [1/4, 3/4]],
             ])
models = {"transition_matrix": M}
result = find_ergodic_distribution(models)

In [ ]:
fastcore.test.test_eq(result['ergodic'],
                      np.array([[1/2, 1/2]]))

In [ ]:
# #| hide
# # Here is some code which illustrates how one could use sympy to find the relevant eigenvectors 
# # using symbolic methods (but please note that even sympy must resort to numerical methods if
# # the matrices are bigger than 5 by 5 in size, due to the fundamental lack of exact solutions to 
# # polynomial equations with order greater than 5)
# import sympy
# for m in M:
#     # Sympy needs integers or expressions to work
#     # Integers is usually safer
#     m = np.array(1000 * m, dtype=int)
#     M_symbolic = sympy.Matrix(m)
#     for result in M_symbolic.eigenvects():
#         lamda, multiplicity, evs = result
        
#         # print("lambda: " , lamda,
#         #           "multiplicity: ", multiplicity,
#         #           "eigenvectors: ", evs)

Here is another quick illustrative example.

Let our transition matrix, $M$ be

\begin{equation}
M = \begin{pmatrix}
\frac{3}{4} & \frac{1}{4} \\
\frac{3}{4} & \frac{1}{4} \\
\end{pmatrix}
\end{equation}

$M^T$ is a stochastic matrix. It is easy to verify that $[\frac{3}{4}, \frac{1}{4}]$ is the normalised eigenvector with eigenvalue 1.


In [ ]:
M = np.array([[[3/4, 1/4],
               [3/4, 1/4]],
             ])
models = {"transition_matrix": M}
result = find_ergodic_distribution(models)

In [ ]:
fastcore.test.test_eq(result['ergodic'],
                      np.array([[3/4, 1/4]]))

In [ ]:
show_doc(find_ergodic_distribution)

---

[source](https://github.com/PaoloBova/gh-pages-example/blob/main/gh_pages_example/methods.py#L113){target="_blank" style="float:right; font-size:smaller"}

### find_ergodic_distribution

>      find_ergodic_distribution (models:dict)

Find the ergodic distribution of a markov chain with the
given transition matrix.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| models | dict | A dictionary that contains the parameters in `ModelTypeEGT` |

### Run full markov chain algorithm

Finally, here is a helper function to both build the transition matrix for the model and find its ergodic distribution.

In [ ]:
#| export
def markov_chain(models:dict # A dictionary that contains the parameters in `ModelTypeEGT`
                ):
    """Find the ergodic distribution of the evolutionary
    game given by each model in models."""
    return thread_macro(models,
                        build_transition_matrix,
                        find_ergodic_distribution)

In [ ]:
show_doc(markov_chain)

---

[source](https://github.com/PaoloBova/gh-pages-example/blob/main/gh_pages_example/methods.py#L132){target="_blank" style="float:right; font-size:smaller"}

### markov_chain

>      markov_chain (models:dict)

Find the ergodic distribution of the evolutionary
game given by each model in models.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| models | dict | A dictionary that contains the parameters in `ModelTypeEGT` |

# References

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()